In [1]:
import pandas as pd
import datetime
from geopy.geocoders import Nominatim

# Transformar partidas -> estadios

In [2]:
partidas = pd.read_csv('../data/raw/campeonato-brasileiro-full.csv')
partidas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  8785 non-null   int64 
 1   rodata              8785 non-null   int64 
 2   data                8785 non-null   object
 3   hora                8785 non-null   object
 4   mandante            8785 non-null   object
 5   visitante           8785 non-null   object
 6   formacao_mandante   3810 non-null   object
 7   formacao_visitante  3810 non-null   object
 8   tecnico_mandante    4175 non-null   object
 9   tecnico_visitante   4175 non-null   object
 10  vencedor            8785 non-null   object
 11  arena               8785 non-null   object
 12  mandante_Placar     8785 non-null   int64 
 13  visitante_Placar    8785 non-null   int64 
 14  mandante_Estado     8785 non-null   object
 15  visitante_Estado    8785 non-null   object
dtypes: int64(4), object(12)


In [3]:
estadios_dict = {}

for partida in partidas.iterrows():
    partida = partida[1]
    estadio = partida['arena']
    if estadio not in estadios_dict:
        estadios_dict[estadio] = {"partidas": 1, 
                                  "vitorias_casa": 0, 
                                  "vitorias_fora": 0, 
                                  "empates": 0, 
                                  "data_primeira_partida": partida["data"], 
                                  "data_ultima_partida": partida["data"],
                                  "times_casa": {}}
    else:
        estadios_dict[estadio]["partidas"] += 1

    if partida["vencedor"] == partida["mandante"]:
        estadios_dict[estadio]["vitorias_casa"] += 1
    elif partida["vencedor"] == partida["visitante"]:
        estadios_dict[estadio]["vitorias_fora"] += 1
    else:
        estadios_dict[estadio]["empates"] += 1

    estadios_dict[estadio]["data_ultima_partida"] = partida["data"]
    estadios_dict[estadio]["times_casa"][partida["mandante"]] = estadios_dict[estadio]["times_casa"].get(partida["mandante"], 0) + 1

estadios_pd = pd.DataFrame(estadios_dict).transpose()

In [4]:
estadios_pd = estadios_pd[estadios_pd["partidas"] > 2]
estadios_pd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries,  Brinco de Ouro to Estádio Independência
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   partidas               114 non-null    object
 1   vitorias_casa          114 non-null    object
 2   vitorias_fora          114 non-null    object
 3   empates                114 non-null    object
 4   data_primeira_partida  114 non-null    object
 5   data_ultima_partida    114 non-null    object
 6   times_casa             114 non-null    object
dtypes: object(7)
memory usage: 7.1+ KB


In [5]:
import time

geolocator = Nominatim(user_agent="analise-visual-brasileirao")
lat = []
lon = []

for index, estadio in estadios_pd.iterrows():
    print(f"Procurando {estadio.name}...")
    location = None
    for attempt in range(3):
        try:
            location = geolocator.geocode(estadio.name)
            if location:
                break
        except Exception as e:
            print(f"Erro na tentativa {attempt+1}: {e}")
            time.sleep(1)
    if location:
        lat.append(location.latitude)
        lon.append(location.longitude)
        print(f"achou {location.latitude}, {location.longitude}")
    else:
        print("não achou")
        lat.append("Unk")
        lon.append("Unk")

estadios_pd["latitude"] = lat
estadios_pd["longitude"] = lon
estadios_pd.to_csv('../data/analise_estadios/estadios.csv', index_label='estadio')

Procurando  Brinco de Ouro...
achou -22.9095729, -47.0433697
Procurando  Arena da Baixada...
achou -12.1266213, -45.0113082
Procurando  Maracanã...
achou -22.9121619, -43.2311861
Procurando  Serra Dourada...
achou -12.758418, -43.9500907
Procurando  Beira Rio...
achou -30.0654898, -51.2349178
Procurando  Heriberto Hulse...
achou -28.6843232, -49.3676706
Procurando  Alfredo Jaconi...
achou -29.162264, -51.1764503
Procurando  Castelão...
achou -3.806742, -38.5216794
Procurando  Mineirão...
achou -19.8659115, -43.9710434
Procurando  Barradão...
achou -12.9190959, -38.4271321
Procurando  Vila Belmiro...
achou -23.9513973, -46.3374426
Procurando  Pacaembu...
achou -21.566243, -51.2633114
Procurando  Couto Pereira...
achou 40.87741, -8.4946954
Procurando  Olímpico...
achou 3.4220246, -76.5324657
Procurando  Fonte Nova...
achou -12.9787926, -38.5035601
Procurando  Orlando Scarpelli...
achou -27.5855462, -48.5864658
Procurando  Mangueirão...
achou -1.3811727, -48.4439318
Procurando  Moisés Luc

In [39]:
estadios_pd.to_csv('../data/analise_estadios/estadios.csv', index_label='estadio')

# Exploração estadios

In [1]:
import pandas as pd

In [4]:
estadios = pd.read_csv('../data/processed/estadios_historico.csv', index_col='estadio')
estadios["data_primeira_partida"] = pd.to_datetime(estadios["data_primeira_partida"], format="%d/%m/%Y")
estadios["data_ultima_partida"] = pd.to_datetime(estadios["data_ultima_partida"], format="%d/%m/%Y")
estadios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries, Brinco de Ouro to Estádio Independência
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   partidas               114 non-null    int64         
 1   vitorias_casa          114 non-null    int64         
 2   vitorias_fora          114 non-null    int64         
 3   empates                114 non-null    int64         
 4   data_primeira_partida  114 non-null    datetime64[ns]
 5   data_ultima_partida    114 non-null    datetime64[ns]
 6   times_casa             114 non-null    object        
 7   latitude               114 non-null    object        
 8   longitude              114 non-null    object        
dtypes: datetime64[ns](2), int64(4), object(3)
memory usage: 8.9+ KB


In [5]:
estadios.head()

,partidas,vitorias_casa,vitorias_fora,empates,data_primeira_partida,data_ultima_partida,times_casa,latitude,longitude
estadio,,,,,,,,,
Brinco de Ouro,66,30,14,22,2003-03-29,2010-11-28,"{'Guarani': 65, 'Corinthians': 1}",-22.9095729,-47.0433697
Arena da Baixada,120,70,17,33,2003-03-29,2011-12-04,"{'Athletico-PR': 119, 'Parana': 1}",-12.1266213,-45.0113082
Maracanã,669,341,140,188,2003-03-30,2024-12-08,"{'Flamengo': 281, 'Fluminense': 296, 'Vasco': ...",-22.9121619,-43.2311861
Serra Dourada,235,115,61,59,2003-03-30,2013-12-08,"{'Goias': 179, 'Athletico-PR': 1, 'Atletico-GO...",-12.758418,-43.9500907
Beira Rio,200,117,36,47,2003-03-30,2012-11-25,{'Internacional': 200},-30.0654898,-51.2349178


In [ ]:
estadios[estadios["data_ultima_partida"].dt.year == 2024].sort_values(by='partidas', ascending=False)

,partidas,vitorias_casa,vitorias_fora,empates,data_primeira_partida,data_ultima_partida,times_casa
estadio,,,,,,,
Maracanã,669,341,140,188,2003-03-30,2024-12-08,"{'Flamengo': 281, 'Fluminense': 296, 'Vasco': ..."
Mineirão,476,255,105,116,2003-03-30,2024-12-04,"{'Cruzeiro': 290, 'Atletico-MG': 184, 'America..."
Couto Pereira,297,140,85,72,2003-04-05,2024-06-22,"{'Coritiba': 290, 'Parana': 4, 'Athletico-PR':..."
Orlando Scarpelli,220,96,54,70,2003-04-06,2024-06-19,"{'Figueirense': 216, 'Parana': 1, 'Flamengo': ..."
Neo Química Arena,204,111,26,67,2014-05-18,2024-12-03,"{'Corinthians': 203, 'Flamengo': 1}"
Barradão,202,90,60,52,2003-03-30,2024-12-04,{'Vitoria': 202}
Arena Castelão,202,92,47,63,2018-04-22,2024-12-08,"{'Ceara': 90, 'Fortaleza': 112}"
Arena do Grêmio,193,116,37,40,2013-06-05,2024-12-08,{'Gremio': 193}
Allianz Parque,165,103,28,34,2014-11-19,2024-12-08,{'Palmeiras': 165}
